In [1]:
### lightgbm -submit10

In [2]:
###Carga de librerias, lectura del dataset

In [3]:
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
#Modificar dimensiones predeterminadas de visibilidad de columnas y filas
x = 52
pd.set_option('display.max_columns', x) #Maximo de columnas a ver
pd.set_option('display.max_rows', x) #Maximo de Filas a ver

In [5]:
path_dataset = 'train.csv'
train = pd.read_csv(path_dataset)

In [6]:
path_dataset = 'test.csv'
test = pd.read_csv(path_dataset)

In [7]:
train.head()

,Opportunity_ID,Region,Billing_Country,Account_Type,Opportunity_Type,Account_Owner,Account_Name,Conversión_USD,Delivery_Quarter,TRF,Stage,promedio_preciov_region,promedio_preciov_paises,promedio_preciov_Account,promedio_preciov_Oportunity,promedio_preciov_vendedor,Wilsonxproporcion_region,Wilsonxproporcion_paises,Wilsonxproporcion_vendedor
0,0,EMEA,Netherlands,Account_Type_2,Opportunity_Type_1,Person_Name_51,Account_Name_619,6305214,Q2,10,Closed Lost,246888.239174,0.000000,0.000000,0.000000,0.000000,0.514817,0.513767,0.325851
1,1,EMEA,Netherlands,Account_Type_2,Opportunity_Type_1,Person_Name_51,Account_Name_619,57673,Q1,0,Closed Won,246888.239174,177583.146032,125034.170409,133491.432403,85165.046512,0.514817,0.513767,0.325851
2,2,Americas,United States,Account_Type_5,Opportunity_Type_1,Person_Name_64,Account_Name_1794,83865,Q1,0,Closed Won,422783.847183,432143.813675,958150.887348,133491.432403,75357.668421,0.445687,0.448366,0.616299
3,3,Americas,United States,Account_Type_5,Opportunity_Type_19,Person_Name_8,Account_Name_1201,7421881,Q1,14,Closed Lost,422783.847183,0.000000,0.000000,0.000000,0.000000,0.445687,0.448366,0.561500
4,4,Americas,United States,Account_Type_5,Opportunity_Type_19,Person_Name_8,Account_Name_1201,13357192,Q1,25,Closed Lost,422783.847183,0.000000,0.000000,0.000000,0.000000,0.445687,0.448366,0.561500


In [8]:
train.head()

,Opportunity_ID,Region,Billing_Country,Account_Type,Opportunity_Type,Account_Owner,Account_Name,Conversión_USD,Delivery_Quarter,TRF,Stage,promedio_preciov_region,promedio_preciov_paises,promedio_preciov_Account,promedio_preciov_Oportunity,promedio_preciov_vendedor,Wilsonxproporcion_region,Wilsonxproporcion_paises,Wilsonxproporcion_vendedor
0,0,EMEA,Netherlands,Account_Type_2,Opportunity_Type_1,Person_Name_51,Account_Name_619,6305214,Q2,10,Closed Lost,246888.239174,0.000000,0.000000,0.000000,0.000000,0.514817,0.513767,0.325851
1,1,EMEA,Netherlands,Account_Type_2,Opportunity_Type_1,Person_Name_51,Account_Name_619,57673,Q1,0,Closed Won,246888.239174,177583.146032,125034.170409,133491.432403,85165.046512,0.514817,0.513767,0.325851
2,2,Americas,United States,Account_Type_5,Opportunity_Type_1,Person_Name_64,Account_Name_1794,83865,Q1,0,Closed Won,422783.847183,432143.813675,958150.887348,133491.432403,75357.668421,0.445687,0.448366,0.616299
3,3,Americas,United States,Account_Type_5,Opportunity_Type_19,Person_Name_8,Account_Name_1201,7421881,Q1,14,Closed Lost,422783.847183,0.000000,0.000000,0.000000,0.000000,0.445687,0.448366,0.561500
4,4,Americas,United States,Account_Type_5,Opportunity_Type_19,Person_Name_8,Account_Name_1201,13357192,Q1,25,Closed Lost,422783.847183,0.000000,0.000000,0.000000,0.000000,0.445687,0.448366,0.561500


In [9]:
###Preprocesamiento

In [10]:
train['Stage']= (train['Stage'] == 'Closed Won') +0

In [11]:
for column in train.columns:
    if train[column].dtype == 'int32':
        train[column] = train[column].astype('float64')
print(train.dtypes)

Opportunity_ID                   int64
Region                          object
Billing_Country                 object
Account_Type                    object
Opportunity_Type                object
Account_Owner                   object
Account_Name                    object
Conversión_USD                   int64
Delivery_Quarter                object
TRF                              int64
Stage                          float64
promedio_preciov_region        float64
promedio_preciov_paises        float64
promedio_preciov_Account       float64
promedio_preciov_Oportunity    float64
promedio_preciov_vendedor      float64
Wilsonxproporcion_region       float64
Wilsonxproporcion_paises       float64
Wilsonxproporcion_vendedor     float64
dtype: object


In [12]:
### Target meaning encoding

In [13]:
# Import KFold
from sklearn.model_selection import KFold

In [14]:
def test_mean_target_encoding(train, test, target, categorical, alpha=5):
    # Calculate global mean on the train data
    global_mean = train[target].mean()
    
    # Group by the categorical feature and calculate its properties
    train_groups = train.groupby(categorical)
    category_sum = train_groups[target].sum()
    category_size = train_groups.size()
    
    # Calculate smoothed mean target statistics
    train_statistics = (category_sum + global_mean * alpha) / (category_size + alpha)
   
    # Apply statistics to the test data and fill new categories
    test_feature = test[categorical].map(train_statistics).fillna(global_mean)
    return test_feature.values

In [15]:
def train_mean_target_encoding(train, target, categorical, alpha=5):
    # Create 5-fold cross-validation
    kf = KFold(n_splits=5, random_state=123, shuffle=True)
    train_feature = pd.Series(index=train.index)
    
    # For each folds split
    for train_index, test_index in kf.split(train):
        cv_train, cv_test = train.iloc[train_index], train.iloc[test_index]
      
        # Calculate out-of-fold statistics and apply to cv_test
        cv_test_feature = test_mean_target_encoding(cv_train, cv_test, target, categorical, alpha)
        
        # Save new feature for this particular fold
        train_feature.iloc[test_index] = cv_test_feature       
    return train_feature.values

In [16]:
def mean_target_encoding(train, test, target, categorical, alpha=5):
  
    # Get the train feature
    train_feature = train_mean_target_encoding(train, target, categorical, alpha)
  
    # Get the test feature
    test_feature = test_mean_target_encoding(train, test, target, categorical, alpha)
    
    # Return new features to add to the model
    return train_feature, test_feature

In [17]:
columns = ['Region', 'Billing_Country', 'Account_Type','Opportunity_Type', 'Account_Owner', 'Account_Name','Delivery_Quarter']
for column in columns:     
    train[column], test[column] = mean_target_encoding(train=train,
                                                                test=test,
                                                                target='Stage',
                                                                categorical=column,
                                                                alpha=10)

<ipython-input-15-5785e70fe0cb>:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  train_feature = pd.Series(index=train.index)


In [18]:
###Seleccion de Features

In [19]:
train.columns

Index(['Opportunity_ID', 'Region', 'Billing_Country', 'Account_Type',
       'Opportunity_Type', 'Account_Owner', 'Account_Name', 'Conversión_USD',
       'Delivery_Quarter', 'TRF', 'Stage', 'promedio_preciov_region',
       'promedio_preciov_paises', 'promedio_preciov_Account',
       'promedio_preciov_Oportunity', 'promedio_preciov_vendedor',
       'Wilsonxproporcion_region', 'Wilsonxproporcion_paises',
       'Wilsonxproporcion_vendedor'],
      dtype='object')

In [20]:
features = ['Region', 'Billing_Country', 'Account_Type',
       'Opportunity_Type', 'Account_Owner', 'Account_Name', 'Conversión_USD',
       'Delivery_Quarter', 'TRF', 'Wilsonxproporcion_region', 'Wilsonxproporcion_paises',
       'Wilsonxproporcion_vendedor']

In [21]:
### Modelos

In [22]:
X = train.drop(['Stage'], axis=1)
y = train['Stage']

In [23]:
###Separando en conjuntos de entrenamiento y de testeo¶

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

In [25]:
###Entrenando los modelos

In [ ]:

###LightGBM

In [ ]:
objective: regression, binary, multiclass, etc (regression)
num_iterations: iteraciones de boosting (100)
learning_rate: tasa de aprendizaje (0.1)
num_leaves: cantidad máxima de hojas en un árbol (31)
max_depth: profundidad máxima (-1)
min_data_in_leaf: cantidad mínima de datos en una hoja (20)
feature_fraction: fracción de los features a utilizar en cada árbol (1.0)
max_bin: cantidad máxima de bins a utilizar para feature continua (255)
categorical_feature: indica cuales son las features categóricas.
metric: métrica a evaluar en el set de test.

In [26]:
import lightgbm as lgb

In [27]:
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import cross_val_score

In [29]:
lgb_clf = lgb.LGBMClassifier()
param_dist = [{"max_depth": [5,10,15],
              "learning_rate" : [0.1,0.15,0.3],
              "num_leaves": [32,150,200],
              "n_estimators": [300,400,1000], 
               'objective' : ['binary']
             }]

In [30]:
grid_search1 = GridSearchCV(lgb_clf, param_grid=param_dist, cv = 3, scoring="neg_log_loss", return_train_score=True).fit(X_train[features], y_train)
                                   

In [31]:
grid_search

GridSearchCV(cv=3, estimator=LGBMClassifier(),
             param_grid=[{'learning_rate': [0.1, 0.15, 0.3],
                          'max_depth': [5, 10, 15],
                          'n_estimators': [300, 400, 1000],
                          'num_leaves': [32, 150, 200],
                          'objective': ['binary']}],
             return_train_score=True, scoring='neg_log_loss')

In [32]:
y_pred=grid_search.predict_proba(X_test[features])

In [33]:
y_pred_opt = y_pred

array([[0.13896669, 0.86103331],
       [0.98252858, 0.01747142],
       [0.96513926, 0.03486074],
       ...,
       [0.29305633, 0.70694367],
       [0.50673727, 0.49326273],
       [0.27968992, 0.72031008]])

In [34]:
grid_search.cv_results_['mean_test_score']

array([-0.43037026, -0.43037026, -0.43037026, -0.43527594, -0.43527594,
       -0.43527594, -0.4797863 , -0.4797863 , -0.4797863 , -0.44117815,
       -0.47549755, -0.47284701, -0.45235544, -0.50628674, -0.50391441,
       -0.54040823, -0.74153996, -0.74055279, -0.43662997, -0.53730375,
       -0.54346183, -0.44995841, -0.5984138 , -0.60903069, -0.54216804,
       -0.9350538 , -0.96142911, -0.43774985, -0.43774985, -0.43774985,
       -0.44701076, -0.44701076, -0.44701076, -0.53173033, -0.53173033,
       -0.53173033, -0.46096382, -0.53833029, -0.52954607, -0.48174999,
       -0.59173856, -0.58507924, -0.61719239, -0.94177167, -0.93386242,
       -0.46303715, -0.63852808, -0.65191725, -0.48182874, -0.73510081,
       -0.76193121, -0.62400103, -1.08800463, -1.09108125, -0.48802781,
       -0.48802781, -0.48802781, -0.51727215, -0.51727215, -0.51727215,
       -0.69759361, -0.69759361, -0.69752345, -0.54695684, -0.75638397,
       -0.74141803, -0.59491035, -0.87954774, -0.86008584, -0.87

In [35]:
grid_search.best_params_

{'learning_rate': 0.1,
 'max_depth': 5,
 'n_estimators': 300,
 'num_leaves': 32,
 'objective': 'binary'}

In [37]:
optimised_lgb= grid_search.best_estimator_

In [38]:
optimised_lgb

LGBMClassifier(max_depth=5, n_estimators=300, num_leaves=32, objective='binary')

In [39]:
# Import log_loss from sklearn
from sklearn.metrics import log_loss

In [40]:
y_opt_pred = y_pred

In [41]:
print('Sklearn LogLoss: {:.5f}'.format(log_loss(y_test, y_opt_pred)))

Sklearn LogLoss: 0.43709


In [43]:
lgb_clf = lgb.LGBMClassifier()
param_dist = [{"max_depth": [3,5,10,15],
              "learning_rate" : [0.1,0.15,0.3],
              "num_leaves": [10,20,32,150,200],
              "n_estimators": [50,100,300,400,1000], 
               'objective' : ['binary']
             }]

In [44]:
grid_search = GridSearchCV(lgb_clf, param_grid=param_dist, cv = 3, scoring="neg_log_loss", return_train_score=True).fit(X_train[features], y_train)

In [45]:
optimised_lgb= grid_search.best_estimator_

In [46]:
optimised_lgb

LGBMClassifier(max_depth=15, n_estimators=50, num_leaves=150,
               objective='binary')

In [47]:
y__opt_pred=optimised_lgb.predict_proba(X_test[features])

print('Sklearn LogLoss: {:.5f}'.format(log_loss(y_test, y_opt_pred)))

In [ ]:
Your submission scored 0.53635, which is not an improvement of your best score. Keep trying!

In [49]:
preds_test= optimised_lgb.predict_proba(test[features])

In [50]:
preds_test=preds_test[:, 1]

In [65]:
test['Target']=preds_test

In [66]:
test['Target'].shape

(2551,)

In [67]:
sub= test[['Opportunity_ID','Target']]

In [68]:
sub1=sub.groupby(['Opportunity_ID'], as_index=False).mean()

In [69]:
sub1[['Opportunity_ID', 'Target']].to_csv('submission10.csv', index=False)

In [ ]:
Your submission scored 0.53635, which is not an improvement of your best score. Keep trying!

In [70]:
test = test.drop(['Target'], axis=1)